In [60]:
import pandas as pd
import re

In [61]:
# Importing our files, lets do arrivals first:

arr_df = pd.read_excel("/Users/alexander/Work/Ironhack/Week12/Project/arrivals.xlsx")

# arr_df.sample(10)

print(arr_df.shape)

no_of_recs = arr_df.shape[0]

print('we have',  no_of_recs,  'unique flight records')

(4949, 19)
we have 4949 unique flight records


In [62]:
# checking and cleaning the data types:

arr_df.dtypes

# Let's convert actual date time to the correct data type, not sure if this'll hurt later

arr_df['Date'] = pd.to_datetime(arr_df['Date'])
arr_df['Scheduled Time'] = pd.to_datetime(arr_df['Scheduled Time'], format='%H:%M').dt.time
arr_df['Actual Arrival Time'] = pd.to_datetime(arr_df['Actual Arrival Time'], format='%H:%M', errors='coerce').dt.time

arr_df.sample(10)

,Departure,Destination,Terminal,Baggage Claim,Gate,Flight Number,Aircraft Type,Plane Reg,Flight Status,Airline,Date,Scheduled Time,Actual Arrival Time,Status,Time Difference (Minutes),Temperature,Wind Speed,Type,Country
4778,Munich,Berlin,1,B1,B17,LH 1926,A321-200,DAISI,Arrived,Lufthansa (LH),2024-07-17,12:05:00,12:45:00,Delayed,40,22.2,11.0,premium,Germany
4775,Thessaloniki,Berlin,1,A1,A36,EJU5028,A320-200 Sharklets,OEIBF,Arrived,easyJet Europe (EJU),2024-07-17,11:40:00,13:20:00,Delayed,100,22.2,11.0,budget,Greece
2281,Chania,Berlin,1,A1,A31,EJU5010,A320-200 Sharklets,OEIZL,Arrived,easyJet Europe (EJU),2024-07-01,20:45:00,22:59:00,Delayed,134,17.7,6.5,budget,Greece
1834,Dublin,Berlin,1,A4,X19,EI 336,A320-200,EICVB,Arrived,Aer Lingus (EI),2024-06-28,21:05:00,20:47:00,Early,-18,23.8,9.8,premium,Ireland
3809,Copenhagen,Berlin,1,A2,A34,EJU5262,A319,OELQJ,Arrived,easyJet Europe (EJU),2024-07-13,09:45:00,09:35:00,Early,-10,18.8,10.3,budget,Denmark
1640,Barcelona,Berlin,2,C2,Z32,VY 1882,A320-200 Sharklets,ECMLE,Arrived,Vueling Airlines (VY),2024-06-28,10:00:00,09:34:00,Early,-26,23.8,9.8,budget,Spain
3185,Riga,Berlin,1,A4,A32,BT 213,A220-300,YLAAO,Arrived,Air Baltic (BT),2024-07-07,19:00:00,19:15:00,Delayed,15,21.1,10.1,premium,Latvia
1252,Bari,Berlin,2,C3,Z31,FR 3310,737-800 Winglets,9HQCY,Arrived,Ryanair (FR),2024-06-26,19:10:00,21:24:00,Delayed,134,28.8,9.2,budget,Italy
1743,Bristol,Berlin,1,A2,X17,EZY2933,A320-200,GEZTM,Arrived,easyJet UK (U2),2024-06-28,16:00:00,17:10:00,Delayed,70,23.8,9.8,budget,UK
4274,Dusseldorf,Berlin,1,A1,A30,EW 9048,A320-200,DABNK,Arrived,Eurowings (EW),2024-07-15,07:30:00,07:14:00,Early,-16,27.0,7.8,budget,Germany


In [63]:
# Some weather data is missing, using the mean for now:

arr_df['Temperature'].fillna(arr_df['Temperature'].mean(), inplace=True)
arr_df['Wind Speed'].fillna(arr_df['Wind Speed'].mean(), inplace=True)

In [64]:
#Removing any severe outliers - 4949

arr_df = arr_df[(arr_df['Time Difference (Minutes)'] > -1000) & (arr_df['Time Difference (Minutes)'] < 1000)]

print(arr_df.shape)

print('We removed', no_of_recs-arr_df.shape[0], 'flight records')

(4933, 19)
We removed 16 flight records


In [65]:
# creating a dataframe that captures each row with missing values to see why and what is missing

missing_data = arr_df[arr_df.isnull().any(axis=1)]

#let's see what unique flight status we have aside from cancelled

display(arr_df['Flight Status'].unique())
print(missing_data.shape) # 179 rows missing data, they could feasibly all be cancellations

# Checking non-cancelled flights
non_can_flights = missing_data[missing_data['Flight Status'] != 'Cancelled']
print(non_can_flights.shape)


#checking cancelled flights

can_flights = missing_data[missing_data['Flight Status'] == 'Cancelled']
print(can_flights.shape)

# we seem to have an error on row 203 - it states cancelled but it clearly arrived.

# checking to see if all missing data is just for cancelled flights

print('We have', can_flights.shape[0], 'cancelled flights')

array(['Arrived', 'Cancelled', 'End of Deboarding', 'Deboarding',
       'Delayed', 'Landing', 'Scheduled', 'Diverted'], dtype=object)

(179, 19)
(44, 19)
(135, 19)
We have 135 cancelled flights


In [66]:
# For now lets create a complete DF with the rows with missing values removed

missing_data = arr_df[arr_df.isnull().any(axis=1)]

# Get indices of rows with missing data
missing_data_indices = missing_data.index

# Create a cleaned DF by dropping the rows with missing data
clean_arr_df = arr_df.drop(missing_data_indices)

print(clean_arr_df.isnull().sum()) # Nice!

clean_arr_df.head(20)


Departure                    0
Destination                  0
Terminal                     0
Baggage Claim                0
Gate                         0
Flight Number                0
Aircraft Type                0
Plane Reg                    0
Flight Status                0
Airline                      0
Date                         0
Scheduled Time               0
Actual Arrival Time          0
Status                       0
Time Difference (Minutes)    0
Temperature                  0
Wind Speed                   0
Type                         0
Country                      0
dtype: int64


,Departure,Destination,Terminal,Baggage Claim,Gate,Flight Number,Aircraft Type,Plane Reg,Flight Status,Airline,Date,Scheduled Time,Actual Arrival Time,Status,Time Difference (Minutes),Temperature,Wind Speed,Type,Country
0,Izmir ADB,Berlin,1,A4,N02,XQ 966,737-800 Winglets,TCSUU,Arrived,Sun Express (XQ),2024-06-16,06:30:00,06:03:00,Early,-27,23.8,10.5,budget,Turkey
1,Beijing,Berlin,1,A3,X17,HU 489,A330-300,B303C,Arrived,Hainan Airlines (HU),2024-06-16,06:45:00,07:11:00,Delayed,26,23.8,10.5,premium,China
2,New York EWR,Berlin,1,A2,Y17,UA 962,767-400/400ER,N76062,Arrived,United Airlines (UA),2024-06-16,07:15:00,07:07:00,Early,-8,23.8,10.5,premium,USA
3,Chisinau,Berlin,1,A1,X01,5F 611,737-800 Winglets,URPSF,Arrived,Fly One (FIA),2024-06-16,07:20:00,08:17:00,Delayed,57,23.8,10.5,budget,Moldova
4,Doha,Berlin,1,A4,X07,QR 079,787-9 Dreamliner,A7BHM,Arrived,Qatar Airways (QR),2024-06-16,07:20:00,07:21:00,Delayed,1,23.8,10.5,premium,Qatar
5,Trieste,Berlin,2,C2,Z34,FR 6781,737 MAX 8,9HVUF,Arrived,Ryanair (FR),2024-06-16,07:35:00,07:14:00,Early,-21,23.8,10.5,budget,Italy
6,Geneva,Berlin,1,A3,A34,EZS1589,A320-200,HBJZX,Arrived,easyJet Switzerland (DS),2024-06-16,07:45:00,07:41:00,Early,-4,23.8,10.5,budget,Switzerland
7,New York JFK,Berlin,1,B3,Y02,DL 092,767-300/300ER,N1603,Arrived,Delta Air Lines (DL),2024-06-16,07:50:00,06:36:00,Early,-74,23.8,10.5,premium,USA
8,Marseille,Berlin,2,C1,Z36,FR 3389,737 MAX 8,9HVUQ,Arrived,Ryanair (FR),2024-06-16,07:55:00,07:44:00,Early,-11,23.8,10.5,budget,France
9,Frankfurt,Berlin,1,B4,B03,LH 170,A320-200 Sharklets,DAIUM,Arrived,Lufthansa (LH),2024-06-16,07:55:00,07:39:00,Early,-16,23.8,10.5,premium,Germany


In [67]:
# Export cleaned data as a csv
clean_arr_df.to_csv('cleaned_arrivals_data.csv', index=False)